In [2]:
import pandas as pd
import os
from utils.generar_enfrent import generar_enfrentamientos
from utils.vuelta_grupos import vuelta_gr
from utils.df_enfrentamientos import df_ida_o_vuelta
from utils.enfrentamientos_sinVS import df_partidos
from utils.descomp_tupla_columnas import descomponer_tupla

 #* Ruta a la carpeta 'data'
carpeta = 'data'

In [3]:
def partidos_jornadas(df_partidos, num_jornada, col_local, col_visitante, col_rtdo_local, col_rtdo_visitante):
    df_partidos_jornada = pd.DataFrame()
    #* Aplicando la función a la columna 'datos' y creando nuevas columnas
    df_partidos_jornada[[col_local, col_visitante]] = df_partidos[f'Jornada {num_jornada}'].apply(descomponer_tupla) #* Usamos la función apply() para aplicar una función a una columna de un dataframe
    df_partidos_jornada[col_rtdo_visitante] = 0
    df_partidos_jornada.insert(0, col_rtdo_local, 0)
    
    return df_partidos_jornada

In [4]:
df_grupos = pd.read_excel('sorteo_grupos.xlsx')
grupo_A = list(df_grupos['Grupo A'])
grupo_B = list(df_grupos['Grupo B'])

In [9]:
 #* GRUPO A:
ida_grA = generar_enfrentamientos(grupo_A) #* Generar partidos de ida
vuelta_grA = vuelta_gr(ida_grA) #* Generar partidos de vuelta
df_ida_grA = df_ida_o_vuelta(ida_grA)  #* Generar DF de los partidos de ida
df_vuelta_grA = df_ida_o_vuelta(vuelta_grA, 6) #* El 6 indica el inicio de la jornada de los partidos de vuelta
idx_grA = 'GRUPO A'
calendario_grA = pd.concat([df_ida_grA, df_vuelta_grA], axis = 1) #* Crear dataframe con todos los partidos de ida y vuelta gr1
calendario_grA['GRUPO'] = idx_grA
calendario_grA.set_index('GRUPO', drop=True, append=False, inplace=True) #* Cambiar el indice del DF para distinguir los grupos al juntarlos
#calendario_grA #* Usar para VISUALIZAR LOS PARTIDOS DE IDA Y VUELTA DEL GRUPO 1

#* GRUPO B:
ida_grB = generar_enfrentamientos(grupo_B) #* Generar partidos de ida
vuelta_grB = vuelta_gr(ida_grB) #* Generar partidos de vuelta
df_ida_grB = df_ida_o_vuelta(ida_grB)  #* Generar DF de los partidos de ida
df_vuelta_grB = df_ida_o_vuelta(vuelta_grB, 6) #* El 6 indica el inicio de la jornada de los partidos de vuelta
idx_grB = 'GRUPO B'
calendario_grB = pd.concat([df_ida_grB, df_vuelta_grB], axis = 1) #* Crear dataframe con todos los partidos de ida y vuelta gr2
calendario_grB['GRUPO'] = idx_grB
calendario_grB.set_index('GRUPO', drop=True, append=False, inplace=True) #* Cambiar el indice del DF para distinguir los grupos al juntarlos
#calendario_grB #* Usar para VISUALIZAR LOS PARTIDOS DE IDA Y VUELTA DEL GRUPO 2

In [6]:
 #! JORNADAS DE IFA SIN FORMATO PARA GENERAR  CADA JORNADA
ida_gA = df_partidos(ida_grA)
ida_gB = df_partidos(ida_grB)

In [7]:
ida_gB

,Jornada 1,Jornada 2,Jornada 3,Jornada 4,Jornada 5
0,"(Diego, Coquina)","(Diego, Armada)","(Diego, Gonzo)","(Diego, Papu)","(Diego, Kike)"
1,"(Kike, Armada)","(Coquina, Gonzo)","(Armada, Papu)","(Gonzo, Kike)","(Papu, Coquina)"
2,"(Papu, Gonzo)","(Kike, Papu)","(Coquina, Kike)","(Armada, Coquina)","(Gonzo, Armada)"


## JORNADA ACTUAL -> CAMBIAR DESDE AQUI LO ANTERIOR NO SE MODIFICA:

In [8]:
 #! PARTIDOS DE LA JORNADA 6:
j6_gA = ida_gA
j6_gB = ida_gB['Jornada 6']
grA_jor5 = partidos_jornadas(ida_gA, 5, 'LOCAL', 'VISITANTE', 'RTDO L', 'RTDO V')
grB_jor5 = partidos_jornadas(ida_gB, 5, 'LOCAL.1', 'VISITANTE.1', 'RTDO L.1', 'RTDO V.1')
vacio = pd.DataFrame()
vacio['Jornada'] = ['JOR. 5'] * 3
df_jornada_actual5  = pd.concat([grA_jor5, vacio, grB_jor5], axis = 1)
df_jornada_actual5

KeyError: 'Jornada 6'

In [ ]:
#ruta_archivo = os.path.join(carpeta, 'jornada_5_inicial.xlsx')
# df_jornada_actual5.to_excel(ruta_archivo, index=False)

## PARTIDOS VUELTA JORNADA ACTUAL

In [22]:
# Tomar solo los partidos de la "Jornada actual"
jornada8 = df_vuelta_grA['Jornada 8']
# Separar "Local" y "Visitante" dividiendo la cadena por ' VS '
jornada8_split = jornada8.str.split(' VS ', expand=True)
jornada8_split.columns = ['LOCAL', 'VISITANTE']
jornada8
# Crear el nuevo DataFrame con el formato deseado
jornada_actual = pd.DataFrame({
    'RTDO L': [0] * len(jornada8_split),  # Columna de resultados locales (inicialmente en 0)
    'LOCAL': jornada8_split['LOCAL'],     # Columna de equipos locales
    'VISITANTE': jornada8_split['VISITANTE'],  # Columna de equipos visitantes
    'RTDO V': [0] * len(jornada8_split),  # Columna de resultados visitantes (inicialmente en 0)
    'jornada': ['JOR. 8'] * len(jornada8_split),  # Columna de la jornada (aquí es "JOR. 1")
    # Columnas adicionales para un segundo grupo de partidos
    'RTDO L.1': [0] * len(jornada8_split),
    'LOCAL.1': ['Gonzo', 'Papu', 'Kike'],  # Ejemplo de equipos adicionales (puedes cambiarlos)
    'VISITANTE.1': ['Puche', 'Armada', 'Coquina' ],  # Ejemplo de visitantes adicionales
    'RTDO V.1': [0] * len(jornada8_split)
})

jornada_actual

,RTDO L,LOCAL,VISITANTE,RTDO V,jornada,RTDO L.1,LOCAL.1,VISITANTE.1,RTDO V.1
0,0,Ruso,Palop,0,JOR. 8,0,Gonzo,Puche,0
1,0,Tony,Fale,0,JOR. 8,0,Papu,Armada,0
2,0,Kero,Lope,0,JOR. 8,0,Kike,Coquina,0


In [21]:
df_vuelta_grB

,Jornada 6,Jornada 7,Jornada 8,Jornada 9,Jornada 10
0,Coquina VS Diego,Armada VS Diego,Gonzo VS Diego,Papu VS Diego,Kike VS Diego
1,Armada VS Kike,Gonzo VS Coquina,Papu VS Armada,Kike VS Gonzo,Coquina VS Papu
2,Gonzo VS Papu,Papu VS Kike,Kike VS Coquina,Coquina VS Armada,Armada VS Gonzo


In [23]:
ruta_archivo = os.path.join(carpeta, 'jornada_8_inicial.xlsx')
jornada_actual.to_excel(ruta_archivo, index=False)